In [ ]:
from typing import List
from pathlib import Path
from tqdm import tqdm
from itertools import chain
import pretty_midi as pm
import symusic as sm
import timeit
import os

MIDI_DATASET_NAMES = ('maestro', 'musicnet', 'POP909', 'giantmidi')
ABC_DATASET_NAMES = ('nottingham',)
DATASET_ROOT = "./symusic_benchmark_datasets"

def is_valid(f: str):
    try:
        sm.Score(f)
        return True
    except:
        return False

MIDI_DATASET = list(chain(
    *list(list(filter(
        is_valid, 
        map(str, Path(DATASET_ROOT).joinpath(name).rglob('*.mid*'))
    )) for name in MIDI_DATASET_NAMES)
))
print(f"Total MIDI Num: {len(MIDI_DATASET)}")

example_p = "xiyangxiaogu.mid"

## Speed Test (symusic)

In [ ]:
for p in tqdm(MIDI_DATASET):
    _ = sm.Score(str(p))

## Multiple Types of Time Unit Support

In [ ]:
example_tick = sm.Score(example_p, ttype='tick') # default
print(f"Note repr full: \n{example_tick.tracks[0].notes[1]}\n")

print("Load as Time Unit [Tick]")
print(example_tick.tracks[0].notes, end="\n\n")

example_quarter = example_tick.to('quarter')
print("Convert to Time Unit [Quarter]")
print(example_quarter.tracks[0].notes, end="\n\n")

example_second = example_quarter.to('second')
print("Convert to Time Unit [Second]")
print(example_second.tracks[0].notes, end="\n\n")

example_tick = example_second.to('tick')
print("Convert Back to Time Unit [Tick]")
print(example_tick.tracks[0].notes)

## Easy to Use Numpy Support

Convert the object list to or back from numpy in one step 

In [ ]:
notes_numpy = example_tick.tracks[0].notes.numpy()
notes_numpy

In [ ]:
example_tick.time_signatures.numpy()

In [ ]:
sm.Note.from_numpy(**notes_numpy)

## Build-in Synthesizer

A simple and fast build-in sythesizer without other dependencies

Get the wave audio as numpy array as soon as possible

In [ ]:
from IPython.display import Audio

sythesizer = sm.Synthesizer(sample_rate=44100)
audio = sythesizer.render(example_tick, stereo=True)
print(type(audio), audio.shape)
Audio(audio, rate=44100)

## PianoRoll Support

fast pianoroll matrix conversion

In [ ]:
from matplotlib import pyplot as plt

s = example_tick
track = s.tracks[0]
pianoroll = track.pianoroll(modes=["onset", "frame", "offset"], pitch_range=[0, 128], encode_velocity=False)
print(type(pianoroll), pianoroll.shape)

# this will show the onset and frame of the piano roll in one figure
pianoroll = pianoroll[0] + pianoroll[1]
plt.imshow(pianoroll, aspect="auto")
plt.ylim(0, 128)
plt.ylabel('Pitch')
plt.xlabel("Time [Tick]")
plt.show()

## Chained Batch Operation

In [ ]:
s = example_tick\
     .copy(deep=True)\
     .shift_time(1000, inplace=True)\
     .shift_pitch(12)\
     .clip(0, 10000, clip_end=True)

In [ ]:
track = s.tracks[0]
pianoroll = track.pianoroll(modes=["onset", "frame"], pitch_range=[0, 128], encode_velocity=False)
print(type(pianoroll), pianoroll.shape)

# this will show the onset and frame of the piano roll in one figure
pianoroll = pianoroll[0] + pianoroll[1]
plt.imshow(pianoroll, aspect="auto")
plt.ylim(0, 128)
plt.ylabel('Pitch')
plt.xlabel("Time [Tick]")
plt.show()